In [ ]:
#code adapted from https://stackoverflow.com/a/37616966
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from scipy.stats._continuous_distns import _distn_names
from csv import writer

dist_names = ['norm','arcsine','beta','bradford','chi','chi2','cosine','expon','f',
              'gamma','invgauss','laplace','t','pareto','pearson3','skewnorm','uniform']

# Create models from data
def best_fit_distribution(data, bins=200):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Best holders
    best_distributions = []

    # Estimate distribution parameters from data
    for ii, distribution in enumerate([d for d in dist_names]):

        #print("{:>3} / {:<3}: {}".format( ii+1, len(dist_names), distribution ))

        distribution = getattr(st, distribution)

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                
                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]
                
                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))
                
                # identify if this distribution is better
                best_distributions.append((distribution, params, sse))
        
        except Exception:
            pass

    
    return sorted(best_distributions, key=lambda x:x[2])

def make_distribution(dist,param):
    if dist == "norm":
        return st.norm.rvs(size=len(df),loc =param[0],scale = param[1])
    elif dist == "arcsine":
        return st.arcsine.rvs(size=len(df),loc =param[0],scale = param[1])
    elif dist == "beta":
        print(a_in)
        print("beta replaced with norm")
        return st.beta.rvs(size=len(df),a=param[0],b=param[1],loc =param[2],scale = param[3])
    elif dist == "bradford":
        return st.bradford.rvs(size=len(df),c=param[0],loc = param[1], scale = param[2])
    elif dist == "chi":
        return st.chi.rvs(size=len(df),df = param[0],loc = param[1], scale = param[2])
    elif dist == "chi2":
        return st.chi2.rvs(size=len(df),df = param[0],loc = param[1], scale = param[2])
    elif dist == "cosine":
        return st.cosine.rvs(size=len(df),loc =param[0],scale = param[1])
    elif dist == "expon":
        return st.expon.rvs(size=len(df),loc =param[0],scale = param[1])
    elif dist == "f":
        return st.f.rvs(size=len(df),dfn=param[0],dfd=param[1],loc =param[2],scale = param[3])
    elif dist == "gamma":
        return st.gamma.rvs(size=len(df),a=param[0],loc = param[1], scale = param[2])
    elif dist == "invgauss":
        return st.invgauss.rvs(size=len(df),mu=param[0],loc = param[1], scale = param[2])
    elif dist == "laplace":
        return st.laplace.rvs(size=len(df),loc =param[0],scale = param[1])
    elif dist == "t":
        return st.t.rvs(size=len(df),df=param[0],loc = param[1], scale = param[2])
    elif dist == "pareto":
        return st.pareto.rvs(size=len(df),b=param[0],loc = param[1], scale = param[2])
    elif dist == "pearson3":
        return st.pearson3.rvs(size=len(df),skew=param[0],loc = param[1], scale = param[2])
    elif dist == "skewnorm":
        return st.skewnorm.rvs(size=len(df),a=param[0],loc = param[1], scale = param[2])
    elif dist == "uniform":
        return st.uniform.rvs(size=len(df),loc =param[0],scale = param[1])
    else:
        print("dist not found")


csv_name = 'ExampleInputData.csv' #INPUT csv name here
df = pd.read_csv(csv_name) #import csv
        
df_example = pd.DataFrame(df)
for col in df.columns:
    df_col = df_example[col].dropna()
    try:
       # Find best fit distribution
        best_distibutions = best_fit_distribution(df_col, 200)
        best_dist = best_distibutions[0]
        
        col_to_append = make_distribution(best_dist[0].name,best_dist[1]) #make distribution
        df_example[col]=df_example[col].dropna()
        df_example[col]=pd.DataFrame(col_to_append)
        
    except Exception:
        print(col)

fname = "ExampleData.csv" #csv name  
f = open(fname, "w+") #write over existing csv
f.close()  
with open(fname, 'a', newline='') as f_object: #open in append mode
    writer_object = writer(f_object)
    writer_object.writerow(df.columns)
    for i in df_example.index:
        writer_object.writerow(df_example.iloc[i])

f_object.close()